In [ ]:
import tensorflow as tf
import cv2
import re
import numpy as np
from pytesseract import pytesseract
from skimage.filters import threshold_local
from pytesseract import Output
pytesseract.tesseract_cmd = r'D:/Coding/tesseract/tesseract.exe'


In [32]:
def bw_scanner(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    T = threshold_local(gray, 21, offset=5, method="gaussian")
    return (gray > T).astype("uint8") * 255

In [33]:
def wrap_perspective(img, rect):
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(img, M, (maxWidth, maxHeight))

In [34]:
def get_boxes_in_pixels(detection_boxes, image_shape):
    height, width, _ = image_shape
    boxes_pixel = [(int(box[1] * width), int(box[0] * height),
                    int(box[3] * width), int(box[2] * height))
                   for box in detection_boxes]
    return boxes_pixel

In [ ]:
# Muat Model
model_path = "D:\Coding\VSCode\Fitur2_ORC\export_model\saved_model"
detect_fn = tf.saved_model.load(model_path)

# Proses Gambar
image_path = "./dataset2/train/receipt_00739_png_jpg.rf.f87a56559be6ee953e69f2660be08569.jpg"
image = cv2.imread(image_path)
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]

# Deteksi Objek
detections = detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detection_boxes = detections['detection_boxes']
detection_scores = detections['detection_scores']

# Filter Deteksi dengan Threshold
threshold = 0.8
boxes = detection_boxes[detection_scores >= threshold]

# Konversi Bounding Box ke Piksel
height, width, _ = image.shape
boxes_pixel = [(int(box[1] * width), int(box[0] * height),
                int(box[3] * width), int(box[2] * height))
               for box in boxes]

# Proses Setiap Bounding Box
for i, (x_min, y_min, x_max, y_max) in enumerate(boxes_pixel):

    rect = np.array([
        [x_min, y_min],  
        [x_max, y_min], 
        [x_max, y_max],  
        [x_min, y_max]   
    ], dtype="float32")
    
    warped = wrap_perspective(image, rect)
    bw_result = bw_scanner(warped)

In [37]:
d = pytesseract.image_to_data(bw_result, output_type=Output.DICT)
n_boxes = len(d['level'])
boxes = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])    
    boxes = cv2.rectangle(boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [38]:
extracted_text = pytesseract.image_to_string(image)
print(extracted_text)

Sprite (L) 25. 000
Lemon Tea (L) 000
Extra Jelly Lychee

TOTAL

100, 000
40.000

CASH
GHANGED




In [39]:
def find_amounts(text):
    amounts = re.findall(r'\d+\.\d{2}\b', text)
    
    total_patterns = [
        r'(?:TOTAL|Total|JUMLAH|Jumlah|AMOUNT|Amount)\s*[:]*[Rp$]*\s*([\d.,]+)',
        r'(?:BAYAR|Pay|PAY)[\s:]*[Rp$]*\s*([\d.,]+)',
        r'(?:GRAND\s*TOTAL|Grand Total)[\s:]*[Rp$]*\s*([\d.,]+)',
    ]

    for pattern in total_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            raw_total = match.group(1)
            # Bersihkan format angka (ganti koma dengan titik)
            cleaned_total = raw_total.replace(',', '.')
            try:
                return float(cleaned_total)
            except ValueError:
                continue

    # Jika pola kata kunci tidak cocok, cari angka dengan format spesifik
    amounts = re.findall(r'\d+\.\d{2}\b', text)

    # Konversi angka menjadi float dan hilangkan duplikat
    unique_amounts = list(dict.fromkeys(float(amount) for amount in amounts))

    # Jika hanya satu angka ditemukan, anggap itu sebagai total
    if len(unique_amounts) == 1:
        return unique_amounts[0]

    # Jika tidak ditemukan angka yang cocok, kembalikan pesan default
    return "Tidak Ditemukan"

In [40]:
amounts = find_amounts(extracted_text)
amounts

100.0